# Ramsey-Cass-Koopmans model

Ramsey model from Chapter 2 of D. Romer's *Advanced Economics* looks as follows...

\begin{align}
    \dot{k}(t) =& k(t)^{\alpha} - c(t) - (g + n + \delta)k(t),\\
    \dot{c}(t) =& \Bigg[\frac{1}{\theta} \bigg[\alpha k(t)^{\alpha - 1} - \delta - \rho - \theta g\bigg]\Bigg]c(t)
\end{align}
...with boundary conditions...
\begin{align}
    k(0) =& k_0 \\
    lim_{t\rightarrow \infty} c(t) =& c^*
\end{align}

...where...

* Elasticity of output with respect to capital: $0 < \alpha < 1$
* Coefficient of relative risk aversion: $0 < \theta$
* Discount rate: $0 < \rho$
* Breakeven investment: $0 < g + n + \delta$
* Convergent lifetime utility: $0 < \rho - n - (1 - \theta)g$

Note that we are assuming two things...

1. Cobb-Douglas production
2. Constant Relative Risk Aversion (CRRA) preferences



In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import sympy as sym

import pycollocation

In [3]:
# define some variables
A, K, L = sym.symbols("A, K, L")

In [4]:
# define some initial conditions
A0, L0 = sym.symbols("A0, L0")

# define some production parameters
alpha, delta = sym.symbols("alpha, delta")

# define growth rates
g, n = sym.symbols("g, n")

# define some preference parameters
theta, rho = sym.symbols("theta, rho")

In [5]:
# user defined extensive production function
def extensive_output(A, K, L, **params):
    """Output is a function of technology, A, capital, K, and labor, L."""
    alpha = params['alpha']
    return K**alpha * (A * L)**(1 - alpha)

def flow_utility(C, **params):
    """Flow utility from consumption (per capita)."""
    theta = params['theta']
    return (C**(1 - theta) - 1) / (1 - theta)

def steady_state_capital(A0, delta, g, n, rho, alpha, theta):
    """
    Equilibrium value of capital (per unit effective labor) with
    Cobb-Douglas production and Constant Relative Risk Aversion (CRRA)
    preferences.
    
    """
    return (alpha / (delta + rho + theta * g))**(1 / (1 - alpha))


In [91]:
class RamseyModelLike(object):
    
    def __init__(self, F, u, params):
        self.extensive_output = F
        self.flow_utility = u
        self.params = params
        
    @property
    def _symbolic_params(self):
        """Return a dictionary of symbolic parameters."""
        return {key: sym.symbols(key) for key in self.params.iterkeys()}
    
    @property
    def _symbolic_vars(self):
        """Return a list of symbolic variables."""
        return sym.symbols(['t', 'k', 'c'])
    
    @property
    def c_dot(self):
        """Symbolic expression defining the equation of motion for consumption (per unit effective labor)."""
        return self.equation_motion_consumption(*self._symbolic_vars, **self._symbolic_params)
        
    @property
    def consumption_euler_equation(self):
        """
        Representative household's consumption Euler equation.

        The Euler equation implies that the growth rate of per
        capita consumption is proportional to the difference
        between the real interest rate and the discount rate of
        the representative household.

        """
        epsilon = self.elasticity_intertemporal_substitution
        r = self.real_interest_rate
        consumption_growth = epsilon * (r - self._symbolic_params['rho'])
        return consumption_growth
    
    @property
    def elasticity_intertemporal_substitution(self):
        """
        Elasticity of intertemporal substitution is reciprical of the
        coefficient of relative risk aversion.

        """
        return 1 / self.relative_risk_aversion_coef

    @property
    def k_dot(self):
        """Symbolic expression defining the equation of motion for capital stock (per unit effective labor)."""
        return self.equation_motion_capital(*self._symbolic_vars, **self._symbolic_params)
    
    @property
    def real_interest_rate(self):
        """Return the real interest rate (net depreciation)."""
        Y = self.extensive_output(1, self._symbolic_vars[1], 1,
                                  **self._symbolic_params)
        mpk = self.marginal_product_capital(Y, self._symbolic_vars[1])
        return mpk - self._symbolic_params['delta']
        
    @property
    def relative_risk_aversion_coef(self):
        """Return the coefficient of relative risk aversion."""
        C = sym.symbols("C")
        u = self.flow_utility(C, **self._symbolic_params)
        crra = -((C * sym.diff(u, C, 2)) / sym.diff(u, C, 1))
        return crra
    
    @property
    def rhs(self):
        """Symbolic expressions for the RHS of the system of ODE."""
        return {'k': self.k_dot, 'c': self.c_dot}
    
    def intensive_output(self, k, **params):
        """
        Intensive output is a function of capital (per unit effective labor)
        and model parameters.

        """
        return self.extensive_output(1, k, 1, **params) 
    
    @staticmethod
    def marginal_product_capital(Y, K):
        """Marginal product of capital."""
        return sym.diff(Y, K, 1)

    @staticmethod
    def marginal_product_labor(Y, L):
        """Marginal product of labor."""
        return sym.diff(Y, L, 1)

    def actual_investment(self, k, c, **params):
        """Return investment (per unit effective labor)."""
        return self.intensive_output(k, **params) - c

    @staticmethod
    def break_even_investment(k, delta, g, n, **params):
        """
        Return level of investment (per unit effective labor) required
        to maintain current level of capital (per unit effective labor).

        """
        return (g + n + delta) * k

    def equation_motion_capital(self, t, k, c, A0, delta, g, n, rho, **params):
        """Equation of motion for capital (per unit effective labor)."""
        k_dot = (self.actual_investment(k, c, **params) -
                 self.break_even_investment(k, delta, g, n, **params))
        return k_dot

    @classmethod
    def equation_motion_consumption(cls, t, k, c, A0, delta, g, n, rho, **params):
        """Equation of motion for consumption (per unit effective labor."""    
        # dependence on global variable C and r!
        C, r = sym.symbols("C, r")
        C_growth_rate = cls.consumption_euler_equation(C, r, rho, **params)
        temp = C_growth_rate.subs({C: A0 * sym.exp(g * t) * c,
                                   r: cls.real_interest_rate(k, delta, **params)})
        c_dot = (temp - g) * c
        return c_dot

    def steady_state_capital_locus(k, delta, g, n, **params):
        """
        Locus of points (k, c) such that :math:`\dot{k}=0`.

        Function returns the level of consumption (per unit
        effective labor) required in order to maintain a given
        level of capital (per unit effective labor).

        """
        locus = (intensive_output(k, **params) -
                 break_even_investment(k, delta, g, n, **params))
        return locus

    def steady_state_consumption(self, A0, delta, g, n, rho, **params):
        """Equilibrium value for consumption (per unit effective labor)."""
        k_star = steady_state_capital(A0, delta, g, n, rho, **params)
        c_star = steady_state_capital_locus(k_star, delta, g, n, **params)
        return c_star
            

In [92]:
# finally we need to define some parameters...
params = {'A0': 1.0, 'alpha': 0.33, 'delta': 0.04, 'theta': 1.05, 'rho': 0.05,
          'g': 0.02, 'n': 0.02}

model = RamseyModelLike(extensive_output, flow_utility, params)

In [93]:
model.elasticity_intertemporal_substitution

C**(-theta + 1)*C**(theta - 1)/theta

In [94]:
# create a dictionary representing the RHS of our model
print model.k_dot

-c - k*(delta + g + n) + k**alpha


In [95]:
model.real_interest_rate

alpha*k**alpha/k - delta

In [97]:
model.consumption_euler_equation.simplify()

(alpha*k**alpha - k*(delta + rho))/(k*theta)

...next we do the same for consumption...

\begin{align}
    0 =& k^{*\alpha} - c^* - (g + n + \delta)k^* \\
    c^* =& k^{*\alpha} - (g + n + \delta)k^*
\end{align}

In [ ]:
# call function with numeric inputs and get a numeric result!
model.steady_state_consumption(**params)

In [ ]:
# define some boundary conditions
k0 = 1.0
c_star = steady_state_consumption(A0=A0, alpha=alpha, delta=delta,
                                  theta=theta, rho=rho, g=g, n=n)

bcs = {'lower': [k - k0],
       'upper': [c - c_star]}
print bcs

## Using pyCollocation

In [ ]:
bvp = pycollocation.SymbolicBoundaryValueProblem(dependent_vars=['k', 'c'],
                                                 independent_var='t',
                                                 rhs=rhs,
                                                 boundary_conditions=bcs,
                                                 params=params)

In [ ]:
solver = pycollocation.OrthogonalPolynomialSolver(bvp)

In [ ]:
# now this works for numerical values as well!
steady_state_capital(**params)

In [ ]:
# define our domain of approximation
domain = [0.0, 100.0]

# initial guess for capital
N = 1000
ts = np.linspace(domain[0], domain[1], N)
ks = steady_state_capital(**params) - (steady_state_capital(**params) - k0) * np.exp(-ts)
cs = np.log(ks)

def initial_polynomial(ts, xs, deg, domain):
    """Eventually need some to link to underlying basis functions!"""
    return np.polynomial.Chebyshev.fit(ts, ks, deg, domain)

def initial_coefs(ts, ks, cs, deg, domain):
    """Return dictionary of coefs."""
    capital = initial_polynomial(ts, ks, deg, domain)
    consumption = initial_polynomial(ts, cs, deg, domain)
    coefs = {'k': capital.coef, 'c': consumption.coef}
    return coefs

In [ ]:
coefs_guess = initial_coefs(ts, ks, cs, 35, domain) 
solver.solve(kind="Chebyshev",
             coefs_dict=coefs_guess,
             domain=domain)

In [ ]:
solver.result.x

In [ ]:
# now create a visualizer for plot the solution...
visualizer = pycollocation.Visualizer(solver)

In [ ]:
# plot the solution
visualizer.interpolation_knots = np.linspace(domain[0], domain[1], N)
visualizer.solution.plot()
plt.show()

In [ ]:
# plot the normalized residuals (they should be small everywhere!)
visualizer.normalized_residuals.plot(logy=True)
plt.show()